# FFWIZARD.COM --- Draft Rankings Generator

### Import Libraries

In [1]:
import numpy as np
import pandas as pd
import requests
import StringIO
from sklearn import preprocessing
from pandas import ExcelWriter
from datetime import datetime

### Pull in ADP Data

In [2]:
def pull_adp(ppr, teams):
    
    # pull adp url with ppr and teams params
    url = 'https://fantasyfootballcalculator.com/adp_csv.php?format=' + ppr + '&teams=' + teams
    page_raw = requests.get(url)
    page_txt = str(page_raw.text)[:]

    # pull in text starting where data is...
    bye_char = page_txt.find("Bye") + 4
    page_data = page_txt[bye_char:]

    # set up list and initialize dataframe
    split_txt = page_data.splitlines()
    df = pd.DataFrame(split_txt[0].split(",")).T

    # iterate through lists and form dataframe
    num_players = len(split_txt) - 1
    for i in range(1, num_players):
        new_row = pd.DataFrame(split_txt[i].split(",")).T
        df = pd.concat([df, new_row], axis=0)

    # format dataframe and include relevant info
    df = df.reset_index().iloc[:, [2, 3, 4, 5, 7]]
    df.columns = ["Rank", "Name", "Pos", "Team", "Bye"]
    
    return df

### Load NFL Schedule

In [3]:
# load nfl schedule data (manually entered)
def load_schedule():
    nfl_sched = pd.read_csv('2017_NFL_Schedule.csv')
    return nfl_sched

### Defense Strength Data

In [4]:
def calc_defense_strength(bye_adjustor):

    # load nfl defense strength data (manually entered)
    def_strength_data = pd.read_csv('Defense_Strength.csv')

    #remove 'source' row
    def_strength = def_strength_data.iloc[:-1, :]

    # make weights based on quality of rankings
    def_rank_weights = pd.Series([0.1, 0.5, 0.3, 0.1])
    
    # create weighted defense rankings array based on weights
    num_teams = len(def_strength)
    weighted_def_rank = np.empty(num_teams, dtype=float)
    
    for i in range(0, num_teams):
        weighted_def_rank[i] = def_rank_weights[0] * float(def_strength.iloc[i, 1]) + def_rank_weights[1] * float(def_strength.iloc[i, 2]) + def_rank_weights[2] * float(def_strength.iloc[i, 3]) + def_rank_weights[3] * float(def_strength.iloc[i, 4])
    weighted_def_rank_df = pd.Series(weighted_def_rank)

    # scale weighted defense rankings to have mean of 0 and std dev of 1
    def_rank_scaled = pd.Series(preprocessing.scale(weighted_def_rank_df))

    # create table with team and score info
    defense_score_no_bye = pd.concat([def_strength.loc[:, "Team"], weighted_def_rank_df, def_rank_scaled], axis=1)
    col_names = ["Team", "Def_Score", "Def_Score_Scaled"]
    defense_score_no_bye.columns = col_names

    # add value for bye weeks in table
    bye_array = pd.DataFrame(["BYE", "XXX", bye_adjustor]).T
    bye_array.columns = col_names
    defense_score = pd.concat([defense_score_no_bye, bye_array])

    defense_score = defense_score.reset_index(drop=True)
    
    return defense_score

### Offense Strength Data

In [5]:
def calc_offense_strength(bye_adjustor):

    # load nfl offense strength data (manually entered)
    off_strength_data = pd.read_csv('Offense_Strength.csv')

    # remove 'source' row
    off_strength = off_strength_data.iloc[:-1, :]

    # make weights based on quality of rankings
    off_rank_weights = pd.Series([0.3, 0.1, 0.3, 0.25, 0.05])
    
    # create weighted offense rankings array based on weights
    num_teams = len(off_strength)
    weighted_off_rank = np.empty(num_teams, dtype=float)
    
    for i in range(0, num_teams):
        weighted_off_rank[i] = off_rank_weights[0] * float(off_strength.iloc[i, 1]) + off_rank_weights[1] * float(off_strength.iloc[i, 2]) + off_rank_weights[2] * float(off_strength.iloc[i, 3]) + off_rank_weights[3] * float(off_strength.iloc[i, 4])  + off_rank_weights[4] * float(off_strength.iloc[i, 5])
    weighted_off_rank_df = pd.Series(weighted_off_rank)

    # scale weighted offense rankings to have mean of 0 and std dev of 1
    off_rank_scaled = pd.Series(preprocessing.scale(weighted_off_rank_df))

    # create table with team and score info
    offense_score_no_bye = pd.concat([off_strength.loc[:, "Team"], weighted_off_rank_df, off_rank_scaled], axis=1)
    col_names = ["Team", "Off_Score", "Off_Score_Scaled"]
    offense_score_no_bye.columns = col_names

    # add value for bye weeks in table
    bye_array = pd.DataFrame(["BYE", "XXX", bye_adjustor]).T
    bye_array.columns = col_names
    offense_score = pd.concat([offense_score_no_bye, bye_array])

    offense_score = offense_score.reset_index(drop=True)
    
    return offense_score

### Add Schedule to ADP Ranks

In [6]:
def sched_per_player(df, nfl_sched):

    # create blank table to store opponent names
    opponents = np.empty([len(df), 17], dtype="S3")

    # iterate through each player and every week
    for i in range(0, len(df)):
        for j in range(0, len(nfl_sched)):

            # set weekly schedule for each player based on team
            if (df.loc[i, "Team"] == nfl_sched.loc[j, "Team"]):
                opponents[i, :] = nfl_sched.iloc[j, 1:]
    
    # put schedule into organized dataframe
    opponents_df = pd.DataFrame(opponents)
    opponents_df.columns = ["Opp_1", "Opp_2", "Opp_3", "Opp_4", "Opp_5", "Opp_6", "Opp_7", "Opp_8", "Opp_9", "Opp_10", "Opp_11", "Opp_12", "Opp_13", "Opp_14", "Opp_15", "Opp_16", "Opp_17"]
    
    return opponents_df

### Create Matchup Strength Table by Week

In [7]:
def weekly_matchup_strength(df, opponents_df, offense_score, defense_score):

    # create empty dataframe to store matchup strengths by week
    str_scores = np.empty([len(df), 17], dtype=float)

    
    # iterate through each players matchup by week
    for i in range(0, opponents_df.shape[0]):
        for j in range(0, opponents_df.shape[1]):

            # if the position is a defense, set matchup strengths based opponents' offense...
            if (df.loc[i, "Pos"] == "DEF"):
                for k in range(0, len(offense_score)):
                    if (opponents_df.iloc[i, j] == offense_score.loc[k, "Team"]):
                        str_scores[i, j] = offense_score.loc[k, "Off_Score_Scaled"]

            # ...otherwise, player is an offensive player, so set matchup strengths based on opponents' defense
            else:
                for l in range(0, len(defense_score)):
                    if (opponents_df.iloc[i, j] == defense_score.loc[l, "Team"]):
                        str_scores[i, j] = defense_score.loc[l, "Def_Score_Scaled"]

    # put matchup strengths into organized dataframe
    str_scores_unwgt = pd.DataFrame(str_scores)
    str_scores_unwgt.columns = ["Str_1", "Str_2", "Str_3", "Str_4", "Str_5", "Str_6", "Str_7", "Str_8", "Str_9", "Str_10", "Str_11", "Str_12", "Str_13", "Str_14", "Str_15", "Str_16", "Str_17"]

    return str_scores_unwgt

### Make Week-by-Week Weightings Array (Higher for Early Weeks and Playoffs)

In [8]:
def week_weight_table(first_playoff_week, last_playoff_week, playoff_weight, week_weights_thru_wk13_qb_wr_k_def, week_weights_thru_wk13_rb_te):
    
    # make blank list to store possible playoff weeks, 14 to 17
    playoff_weeks_qb_wr_k_def = []
    playoff_weeks_rb_te = []

    # week 14
    if (first_playoff_week == 14):
        playoff_weeks_qb_wr_k_def.append(playoff_weight)
        playoff_weeks_rb_te.append(playoff_weight)
    else:
        playoff_weeks_qb_wr_k_def.append(week_weights_thru_wk13_qb_wr_k_def[12])
        playoff_weeks_rb_te.append(week_weights_thru_wk13_rb_te[12])

    # week 15
    if (first_playoff_week >= 14):
        playoff_weeks_qb_wr_k_def.append(playoff_weight)
        playoff_weeks_rb_te.append(playoff_weight)
    else:
        playoff_weeks_qb_wr_k_def.append(week_weights_thru_wk13_qb_wr_k_def[12])
        playoff_weeks_rb_te.append(week_weights_thru_wk13_rb_te[12])

    # week 16
    if (last_playoff_week >= 16):
        playoff_weeks_qb_wr_k_def.append(playoff_weight)
        playoff_weeks_rb_te.append(playoff_weight)
    else:
        playoff_weeks_qb_wr_k_def.append(0)
        playoff_weeks_rb_te.append(0)

    # week 17
    if (last_playoff_week == 17):
        playoff_weeks_qb_wr_k_def.append(playoff_weight)
        playoff_weeks_rb_te.append(playoff_weight)
    else:
        playoff_weeks_qb_wr_k_def.append(0)
        playoff_weeks_rb_te.append(0)

    # combine weeks 1 to 13 with playoff weeks
    week_weights_qb_wr_k_def = week_weights_thru_wk13_qb_wr_k_def + playoff_weeks_qb_wr_k_def
    week_weights_rb_te = week_weights_thru_wk13_rb_te + playoff_weeks_rb_te
    
    return week_weights_qb_wr_k_def, week_weights_rb_te

### Apply Week-by-Week Weights to Player Weekly Matchups

In [9]:
def weight_str_scores(df, str_scores_unwgt, week_weights_qb_wr_k_def, week_weights_rb_te, first_playoff_week, last_playoff_week, low_rank_adjustor):

    # pull in adp ranks as floats
    df_rank = df.loc[:, "Rank"]
    df_rank = map(float, df_rank)
    df_rank_array = np.array(df_rank)
    
    # make copy of weekly matchup strengths for each player
    str_scores_wgt = str_scores_unwgt.copy()

    # iterate through each row and multiply matchup strengths by weekly weights
    for i in range(0, len(str_scores_unwgt)):
        
        # if player position is RB or TE, apply weight curve reflecting greater chance of injury
        if (df.loc[i, "Pos"] == "RB") or (df.loc[i, "Pos"] == "TE"):
            wgt_str_scores = np.multiply(np.asarray(str_scores_unwgt.iloc[i, :]), np.asarray(week_weights_rb_te))
            str_scores_wgt.iloc[i, :] = wgt_str_scores
        
        # otherwise apply differenct curve
        else:
            wgt_str_scores = np.multiply(np.asarray(str_scores_unwgt.iloc[i, :]), np.asarray(week_weights_qb_wr_k_def))
            str_scores_wgt.iloc[i, :] = wgt_str_scores
            
                    
        for j in range(first_playoff_week-1, last_playoff_week-1):
            str_scores_wgt.iloc[i, j] = wgt_str_scores[j] * (1 - (df_rank_array[i] / low_rank_adjustor))

    # put weighted matchup strengths into organized dataframe
    str_scores_wgt.columns = ["Wgt_1", "Wgt_2", "Wgt_3", "Wgt_4", "Wgt_5", "Wgt_6", "Wgt_7", "Wgt_8", "Wgt_9", "Wgt_10", "Wgt_11", "Wgt_12", "Wgt_13", "Wgt_14", "Wgt_15", "Wgt_16", "Wgt_17"]
    
    return str_scores_wgt

### Overall Strength Rating Per Player

In [10]:
def player_weighted_sched_str(str_scores_wgt, str_scores_unwgt, overall_sos_weight):

    # set up dummy strength array (weighted)
    str_scores_sum_wgt = str_scores_wgt.iloc[:, 0]

    # populate array with summed scores (weighted)
    for i in range(0, len(str_scores_wgt)):
        str_scores_sum_wgt[i] = str_scores_wgt.iloc[i, :].sum()

    # set up dummy strength array (unweighted)
    str_scores_sum_unwgt = str_scores_unwgt.iloc[:, 0]
    
    # populate array with summed scores (unweighted)
    for j in range(0, len(str_scores_unwgt)):
        str_scores_sum_unwgt[j] = str_scores_unwgt.iloc[j, :].sum()
    
    # scale weighted sos values (mean = 0, var = 1)
    str_scores_scaled_wgt = pd.Series(preprocessing.scale(str_scores_sum_wgt))

    # scale unweighted sos values (mean = 0, var = 1)
    str_scores_scaled_unwgt = pd.Series(preprocessing.scale(str_scores_sum_unwgt))
    
    
    # set up dummy array adjusting out strength of scedule (unscaled)
    str_scores_unscaled = str_scores_unwgt.iloc[:, 0]
    
    # populate array with summed scores (unweighted)
    for k in range(0, len(str_scores_unscaled)):
        str_scores_unscaled[k] = str_scores_scaled_wgt[k] - (overall_sos_weight * str_scores_scaled_unwgt[k])
    
    # scale unweighted sos values (mean = 0, var = 1)
    str_scores_scaled = pd.Series(preprocessing.scale(str_scores_unscaled))
    
    return str_scores_scaled

### Compute Strength of Schedule Score

In [11]:
def ffwiz_score_scaled(df, str_scores_scaled, rank_adder, matchup_strength_scaler, def_rank_penalty, k_rank_penalty):

    # pull in adp ranks and SoS scores
    df_rank = df.loc[:, "Rank"]
    df_rank = map(float, df_rank)
    df_rank_array = np.array(df_rank)
    
    # initialize adjusted rankings
    df_rank_adj = df_rank_array.copy()

    # iterate through the players and add penalty to DEF and K
    for i in range(0, len(df)):
        if df.loc[i, "Pos"] == "DEF":
            df_rank_adj[i] = df_rank_array[i] + rank_adder + def_rank_penalty
        elif df.loc[i, "Pos"] == "PK":
            df_rank_adj[i] = df_rank_array[i] + rank_adder + k_rank_penalty
        else:
            df_rank_adj[i] = df_rank_array[i] + rank_adder

    # adjust adp ranks and SoS scores (to move players up/down appropriate slots)
    str_scores_scaled_adj = matchup_strength_scaler - str_scores_scaled

    # multiply adjusted adp ranks and SoS scores
    df_rank_adj = np.multiply(df_rank_adj, str_scores_scaled_adj)
    ffwiz_score = np.asarray(df_rank_adj)
    
    return(ffwiz_score)

### Overall FFWIZ Rankings

In [12]:
def ffwiz_overall_rankings(df, ffwiz_score):

    # sort strength of schedule scores and put into dataframe
    ffwiz_score_df = pd.DataFrame(ffwiz_score, columns=["FFWIZ_Score"])
    ffwiz_score_sort = ffwiz_score_df.sort_values(by='FFWIZ_Score')
    ffwiz_sort_idx = ffwiz_score_sort.index.values + 1
    ffwiz_sort_df = pd.DataFrame(ffwiz_sort_idx, columns=["FFWIZ"])

    # get ffwiz overall rankings using above dataframe
    ff_sort = ffwiz_sort_df.sort_values(by='FFWIZ')
    ff_sort_idx = ff_sort.index.values + 1
    ff_sort_df = pd.DataFrame(ff_sort_idx, columns=["FFWIZ"])

    # make dataframe for adp rankings
    adp_rank_df = pd.DataFrame(range(1, len(ffwiz_sort_idx)+1), columns=["ADP"])
    adp_values = np.array(range(1, len(ffwiz_sort_idx)+1))

    # calculate difference in ffwiz vs. adp rankings
    diff = pd.DataFrame(np.asarray(adp_rank_df) - np.asarray(ff_sort_df), columns=["DIFF"])

    # make master dataframe and sort
    ffwiz_df = pd.concat([ff_sort_df, adp_rank_df, diff, df.iloc[:, 1:]], axis=1)
    ffwiz_df_overall = ffwiz_df.sort_values(by='FFWIZ').reset_index(drop=True)

    return ffwiz_df_overall

### Positional FFWIZ Rankings

In [13]:
def ffwiz_df_positional(ffwiz_df_overall):

    # FFWIZ
    # sort by position and add FFWIZ and ADP positional rankings
    ffwiz_df_no_pos_rank = ffwiz_df_overall.sort_values(by=['Pos', 'FFWIZ']).reset_index(drop=True)

    # add positional ranks to FFWIZ rankings
    ffwiz_pos_rank_list = [1] #initialize pos list
    ffwiz_rank_counter = 1 #initialize positional rank to 1

    # populate positional rank list
    for k in range(1, len(ffwiz_df_overall)):

        # reset positional rank value if 
        if (ffwiz_df_no_pos_rank.loc[k, 'Pos'] == ffwiz_df_no_pos_rank.loc[k-1, 'Pos']):
            ffwiz_rank_counter = ffwiz_rank_counter + 1
        else:
            ffwiz_rank_counter = 1

        ffwiz_pos_rank_list.append(ffwiz_rank_counter)

    ffwiz_df_pos_rank = pd.DataFrame(ffwiz_pos_rank_list, columns=["FFWIZ_Pos_Rank"]).reset_index(drop=True)


    # ADP
    # sort by position and add and ADP positional rankings
    adp_df_no_pos_rank = ffwiz_df_overall.sort_values(by=['Pos', 'ADP']).reset_index(drop=True)
    adp_pos_rank_table = pd.concat([adp_df_no_pos_rank, ffwiz_df_pos_rank], axis=1)
    adp_pos_sort_ffwiz = adp_pos_rank_table.sort_values(by=['Pos', 'FFWIZ']).reset_index(drop=True)
    adp_pos_rank_array = np.asarray(adp_pos_sort_ffwiz.loc[:, 'FFWIZ_Pos_Rank'])
    adp_pos_rank = pd.DataFrame(adp_pos_rank_array, columns=["ADP_Pos_Rank"])

    # DIFF & CONCATENATE
    ffwiz_pos_rank_array = np.asarray(ffwiz_df_pos_rank.loc[:, 'FFWIZ_Pos_Rank'])
    pos_diff = pd.DataFrame(adp_pos_rank_array - ffwiz_pos_rank_array, columns=["Pos_DIFF"])
    ffwiz_df_pos = pd.concat([ffwiz_df_pos_rank, adp_pos_rank, pos_diff, ffwiz_df_no_pos_rank], axis=1)

    return ffwiz_df_pos

### Generate Overall and Positional Rankings (Fxns of Fxns)

In [14]:
def generate_ffwiz_data(teams, ppr, first_playoff_week, last_playoff_week, week_weights_thru_wk13_qb_wr_k_def, week_weights_thru_wk13_rb_te, playoff_weight, rank_adder, matchup_strength_scaler, overall_sos_weight, def_rank_penalty, k_rank_penalty):
    
    # pull in adp data and schedule based on PPR selection and # of teams in league (FROM USER)
    df = pull_adp(ppr, teams)
    nfl_sched = load_schedule()

    # make dataframe with each players opponents
    opponents_df = sched_per_player(df, nfl_sched)

    # calculate defense and offense strength scores
    defense_score = calc_defense_strength(bye_adjustor)
    offense_score = calc_offense_strength(bye_adjustor)

    # map matchup strengths for each player for every week
    str_scores_unwgt = weekly_matchup_strength(df, opponents_df, offense_score, defense_score)

    # generate weekly weights based on inputted playoff weeks (FROM USER)
    week_weights_qb_wr_k_def, week_weights_rb_te = week_weight_table(first_playoff_week, last_playoff_week, playoff_weight, week_weights_thru_wk13_qb_wr_k_def, week_weights_thru_wk13_rb_te)

    # weight each weekly matchup for each player
    str_scores_wgt = weight_str_scores(df, str_scores_unwgt, week_weights_qb_wr_k_def, week_weights_rb_te, first_playoff_week, last_playoff_week, low_rank_adjustor)

    # get ONE value per player by summing all weighted weekly matchup scores and SCALING so mean=0, var=1
    str_scores_scaled = player_weighted_sched_str(str_scores_wgt, str_scores_unwgt, overall_sos_weight)
    
    # get ffwiz absolute score rankings based on further adjusting of differentials
    ffwiz_score = ffwiz_score_scaled(df, str_scores_scaled, rank_adder, matchup_strength_scaler, def_rank_penalty, k_rank_penalty)

    # get overall ffwiz rankings
    ffwiz_df_overall = ffwiz_overall_rankings(df, ffwiz_score)

    # get positional ffwiz rankings
    ffwiz_df_pos = ffwiz_df_positional(ffwiz_df_overall)
    
    return ffwiz_df_overall, ffwiz_df_pos

### Make List of Dataframe Outputs

In [20]:
def make_dataframes_lists(teams_array, ppr_array, first_playoff_week_array, last_playoff_week_array):

    ffwiz_overall_list = []
    ffwiz_pos_list = []
    tab_name_list = []

    # iterate through all user inputs
    for i in range(0, len(teams_array)):
        for j in range(0, len(ppr_array)):
            for k in range(0, len(first_playoff_week_array)):
                for l in range(0, len(last_playoff_week_array)):
                    
                    #generate rankings
                    ffwiz_df_overall, ffwiz_df_pos = generate_ffwiz_data(teams_array[i], ppr_array[j], first_playoff_week_array[k], last_playoff_week_array[l], week_weights_thru_wk13_qb_wr_k_def, week_weights_thru_wk13_rb_te, playoff_weight, rank_adder, matchup_strength_scaler, overall_sos_weight, def_rank_penalty, k_rank_penalty)

                    ffwiz_overall_list.append(ffwiz_df_overall)
                    ffwiz_pos_list.append(ffwiz_df_pos)
                    
                    #make list of tabnames for excel file
                    tab_name =  teams_array[i] + '_' + str(j) + '_' + str(first_playoff_week_array[k]) + '_' + str(last_playoff_week_array[l])
                    tab_name_list.append(tab_name)
    
                    # print to keep track of progress
                    print(tab_name)
                    print(str(datetime.now()))
    
    return ffwiz_overall_list, ffwiz_pos_list, tab_name_list       

### Make Excel File with Overall and Positional Rankings Tabs

In [16]:
def make_excel(ffwiz_overall_list, ffwiz_pos_list, tab_name_list):

    # make writer object for Excel file
    writer = pd.ExcelWriter('./rankings_data_files/rankings_data_master.xlsx', engine='xlsxwriter')
    
    # iterate through every input permutation
    for i in range(0, len(tab_name_list)):
        
        # generate breakpoints of positions in order to split tabs by position
        idx_list = []
        for j in range(0, len(ffwiz_pos_list[i])):
                if ffwiz_pos_list[i].loc[j, 'FFWIZ_Pos_Rank'] == 1:
                    idx_list.append(j)
        idx_list.append(j) #add END value for indexing

        # create positional dataframes based on breakpoints above
        ffwiz_def = ffwiz_pos_list[i].iloc[idx_list[0]:idx_list[1]]
        ffwiz_k = ffwiz_pos_list[i].iloc[idx_list[1]:idx_list[2]]
        ffwiz_qb = ffwiz_pos_list[i].iloc[idx_list[2]:idx_list[3]]
        ffwiz_rb = ffwiz_pos_list[i].iloc[idx_list[3]:idx_list[4]]
        ffwiz_te = ffwiz_pos_list[i].iloc[idx_list[4]:idx_list[5]]
        ffwiz_wr = ffwiz_pos_list[i].iloc[idx_list[5]:idx_list[6]]

        # create tabs for overall and each position in Excel file
        ffwiz_overall_list[i].to_excel(writer, sheet_name=tab_name_list[i]+'_ALL')
        ffwiz_def.to_excel(writer, sheet_name=tab_name_list[i]+'_DEF')
        ffwiz_k.to_excel(writer, sheet_name=tab_name_list[i]+'_K')
        ffwiz_qb.to_excel(writer, sheet_name=tab_name_list[i]+'_QB')
        ffwiz_rb.to_excel(writer, sheet_name=tab_name_list[i]+'_RB')
        ffwiz_te.to_excel(writer, sheet_name=tab_name_list[i]+'_TE')
        ffwiz_wr.to_excel(writer, sheet_name=tab_name_list[i]+'_WR')

    writer.save()

### Inputs

In [17]:
# GLOBAL INPUTS (constant regardless of user input)
week_weights_thru_wk13_qb_wr_k_def = [1,0.98,0.96,0.94,0.92,0.90,0.88,0.86,0.84,0.82,0.80,0.78,0.76]  #based on games started analysis & trade assumptions
week_weights_thru_wk13_rb_te = [1,0.96,0.92,0.88,0.84,0.80,0.76,0.72,0.68,0.64,0.60,0.56,0.52]  #based on games started analysis & trade assumptions
playoff_weight = 1.2           # was 1.25
rank_adder = 100               # was 100
matchup_strength_scaler = 45   # was 50
overall_sos_weight = 0.5       # was 0.5
low_rank_adjustor = 350        # to adjust-out the importance of playoff weeks for lower-tier players
k_rank_penalty = 5
def_rank_penalty = 2
bye_adjustor = -2.5


# USER INPUTS
teams_array = ['8', '10', '12', '14']
ppr_array = ['standard', 'ppr']
first_playoff_week_array = [14, 15]
last_playoff_week_array = [15, 16, 17]

### Generate Rankings and Make Excel Output (Run Functions!)

In [18]:
ffwiz_overall_list, ffwiz_pos_list, tab_name_list = make_dataframes_lists(teams_array, ppr_array, first_playoff_week_array, last_playoff_week_array)

In [19]:
make_excel(ffwiz_overall_list, ffwiz_pos_list, tab_name_list)

8_0_14_15
2017-07-20 15:32:44.994000
8_0_14_16
2017-07-20 15:32:45.106000
8_0_14_17
2017-07-20 15:32:45.231000
8_0_15_15
2017-07-20 15:32:45.336000
8_0_15_16
2017-07-20 15:32:45.473000
8_0_15_17
2017-07-20 15:32:45.633000
8_1_14_15
2017-07-20 15:32:45.734000
8_1_14_16
2017-07-20 15:32:45.850000
8_1_14_17
2017-07-20 15:32:45.962000
8_1_15_15
2017-07-20 15:32:46.068000
8_1_15_16
2017-07-20 15:32:46.177000
8_1_15_17
2017-07-20 15:32:46.286000
10_0_14_15
2017-07-20 15:32:46.457000
10_0_14_16
2017-07-20 15:32:46.642000
10_0_14_17
2017-07-20 15:32:46.778000
10_0_15_15
2017-07-20 15:32:46.903000
10_0_15_16
2017-07-20 15:32:47.027000
10_0_15_17
2017-07-20 15:32:47.168000
10_1_14_15
2017-07-20 15:32:47.311000
10_1_14_16
2017-07-20 15:32:47.577000
10_1_14_17
2017-07-20 15:32:47.754000
10_1_15_15
2017-07-20 15:32:47.914000
10_1_15_16
2017-07-20 15:32:48.063000
10_1_15_17
2017-07-20 15:32:48.211000
12_0_14_15
2017-07-20 15:32:48.335000
12_0_14_16
2017-07-20 15:32:48.515000
12_0_14_17
2017-07-20 15